# Chapter 5. Spark SQL and DataFrames: Interacting with External Data Sources

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.config("spark.driver.memory", "3g").config("spark.jars", "/home/kolesnik/work/learning/spark/jars/postgresql.jar").config("spark.sql.catalogImplementation","hive").appName("SparkSQLExampleApp").getOrCreate()

## Spark SQL and Apache Hive, User-Defined Functions (UDF)

In [ ]:
from pyspark.sql.types import LongType

# Create and register the `cubed()` user-defined function
def cubed(s):
    return s * s * s
spark.udf.register("cubed", cubed, LongType())

# Generate a temporary view and query it using UDF
spark.range(1, 9).createOrReplaceTempView("udf_test")
spark.sql("select id, cubed(id) as id_cubed from udf_test").show()

In [ ]:
# Pandas UDF
import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the cubed function
def cubed(a: pd.Series) -> pd.Series:
    return a * a * a
cubed_udf = pandas_udf(cubed, returnType=LongType())

In [ ]:
# The function for a pandas_udf executed with local Pandas data
x = pd.Series([1, 2, 3])
print(cubed(x))

In [ ]:
# Create a Spark DataFrame and Execute function as a Spark vectorized UDF
df = spark.range(1, 4)
df_cubed = df.select("id", cubed_udf(df.id).alias("cubed_id"))
df_cubed.show()

## Querying with the Spark SQL Shell, Beeline, and Tableau

### Using the Spark SQL Shell

```
CREATE TABLE people (name STRING, age int);
insert INTO people VALUES ("Michael", NULL);
insert INTO people VALUES ("Andy", 30);
insert INTO people VALUES ("Samantha", 19);
show tables;
select * from people;
select * from people where age < 20;
select * from people where age is null;

```

### Running the Thrift server using only pyspark package
```
park-class org.apache.spark.deploy.SparkSubmit --class org.apache.spark.sql.hive.thriftserver.HiveThriftServer2 spark-internal

beeline

!connect jdbc:hive2://localhost:10000
```
Use `<user>@<host>` for username and blank for password.

## External Data Sources, PostgreSQL

Note that a JAR with the corresponding PostgreSQL JDBC driver was added to the classpath while buiding a `SparkSession` ojbect (see `config("spark.jars", ...)` call).

In [ ]:
(df_cubed
 .write
 .format("jdbc")
 .option("driver", "org.postgresql.Driver")  # This line is missing in the book and leads to the `java.sql.SQLException: No suitable driver` exception
 .option("url", "jdbc:postgresql://localhost:5432/postgres")
 .option("user", "postgres")
 .option("password", "example")
 .option("dbtable", "public.cubed")
 .mode("overwrite")
 .save())

In [ ]:
(spark
 .read
 .format("jdbc")
 .option("driver", "org.postgresql.Driver")  # This line is missing in the book and leads to the `java.sql.SQLException: No suitable driver` exception
 .option("url", "jdbc:postgresql://localhost:5432/postgres")
 .option("user", "postgres")
 .option("password", "example")
 .option("dbtable", "public.cubed")
 .load()).show()

## Higher-Order Functions in DataFrames and Spark SQL

In [ ]:
# Sample data. Series of temperature values in Celsius
schema = "`celsius` array<int>"
t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
t_c = spark.createDataFrame(t_list, schema)
t_c.createOrReplaceTempView("tC")
# Show the DataFrame
t_c.show()

### transform()

In [ ]:
# Calculate Fahrenheit from Celsius for an array of temperatures
spark.sql("select celsius, transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit from tC").show()

In [ ]:
t_c.select("celsius", F.transform("celsius", lambda t: ((t * 9) / 5) + 32).alias("fahrenheit")).show()

### filter()

In [ ]:
# Filter temperatures > 38C for array of temperatures
spark.sql("select celsius, filter(celsius, t -> t > 38) as high from tC").show()

In [ ]:
t_c.select("celsius", F.filter("celsius", lambda t: t > 38).alias("high")).show()

### exists()

In [ ]:
# Is there a temperature of 38C in the array of temperatures
spark.sql("""
SELECT celsius,
exists(celsius, t -> t = 38) as threshold
FROM tC
""").show()

In [ ]:
t_c.select("celsius", F.exists("celsius", lambda x: x == 38).alias("threshold")).show()

### reduce()

In [ ]:
# Calculate average temperature and convert to F
spark.sql("""
SELECT celsius,
reduce(
celsius,
0,
(t, acc) -> t + acc,
acc -> (acc div size(celsius) * 9 div 5) + 32
) as avgFahrenheit
FROM tC
""").show()

There is no a corresponding `reduce()` function in `pyspark.sql.functions`

## Common DataFrames and Spark SQL Operations

In [ ]:
tripdelaysFilePath = "../data/departuredelays.csv"
airportsnaFilePath = "../data/airport-codes-na.txt"

In [ ]:
# Obtain airports data set
airportsna = (spark.read
 .format("csv")
 .options(header="true", inferSchema="true", sep="\t")
 .load(airportsnaFilePath))
airportsna.createOrReplaceTempView("airports_na")

In [ ]:
# Obtain departure delays data set
departureDelays = (spark.read
 .format("csv")
 .options(header="true")
 .load(tripdelaysFilePath))
departureDelays = (departureDelays
 .withColumn("delay", F.expr("CAST(delay as INT) as delay"))
 .withColumn("distance", F.expr("CAST(distance as INT) as distance")))
departureDelays.createOrReplaceTempView("departureDelays")

In [ ]:
# Create temporary small table
foo = (departureDelays
 .filter(F.expr("origin == 'SEA' and destination == 'SFO' and date like '01010%' and delay > 0")))
foo.createOrReplaceTempView("foo")

In [ ]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()
spark.sql("SELECT * FROM foo").show()

### Unions

In [ ]:
# Union of foo and departureDelays creates duplicate entries, because foo rows were taked from departureDelays
bar = departureDelays.union(foo)
bar.createOrReplaceTempView("bar")

In [ ]:
# Show the union (filtering for SEA and SFO in a specific time range)
bar.filter(F.expr("""origin == 'SEA' AND destination == 'SFO' AND date LIKE '01010%' AND delay > 0""")).show()

In [ ]:
# Show the union in SQL
spark.sql("""
SELECT *
FROM bar
WHERE origin = 'SEA'
AND destination = 'SFO'
AND date LIKE '01010%'
AND delay > 0
""").show()

### Joins

In [ ]:
airportsna.show(2)
foo.show(2)

In [ ]:
# Join departure delays data (foo) with airport info (default inner join)
foo.alias("f").join(airportsna.alias("a"), airportsna.IATA == foo.origin).select("a.City", "a.State", "f.date", "f.delay", "f.distance", "f.destination").show()

In [ ]:
# Same in SQL
spark.sql("""
SELECT a.City, a.State, f.date, f.delay, f.distance, f.destination
FROM foo f
JOIN airports_na a
ON a.IATA = f.origin
""").show()

### Windowing

Note that for "create table ... as select" to work we set catalog implementation to "hive" while buiding a `SparkSession` ojbect (see `config("spark.sql.catalogImplementation","hive")` call).

In [ ]:
spark.sql("DROP TABLE IF EXISTS departureDelaysWindow")
spark.sql("""
create table departureDelaysWindow as
select origin, destination, sum(delay) as TotalDelays
  from departureDelays
  where origin in ('SEA', 'SFO', 'JFK')
    and destination in ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
  group by origin, destination
""")

In [ ]:
# For each of the origin airports find the three destinations that experienced the most delays
spark.sql("""
SELECT origin, destination, TotalDelays, rank
  FROM (
    SELECT origin, destination, TotalDelays, dense_rank()
        OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank
      FROM departureDelaysWindow
  )
WHERE rank <= 3
""").show()

More on Spark's window functions [here](https://www.databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html)

### Modifications

In [ ]:
foo.show()

In [ ]:
# Adding new columns
foo2 = foo.withColumn("status", F.expr("case when delay <= 10 then 'On-time' else 'Delayed' end"))
foo2.show()

In [ ]:
# Same transformation using DataFrame API
foo22 = foo.withColumn("status", F.when(foo.delay <= 10, "On-time").otherwise("Delayed"))
foo22.show()

In [ ]:
# Dropping columns
foo3 = foo2.drop("delay")
foo3.show()

In [ ]:
# Renaming columns
foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

In [ ]:
# Pivoting
spark.sql("""
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
  FROM departureDelays
WHERE origin = 'SEA'
""").show(10)

In [ ]:
spark.sql("""
SELECT * FROM (
  SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
    FROM departureDelays WHERE origin = 'SEA'
)
PIVOT (
  CAST(AVG(delay) AS DECIMAL(4, 2)) AS AvgDelay, MAX(delay) AS MaxDelay
  FOR month IN (1 JAN, 2 FEB)
)
ORDER BY destination
""").show()

More on pivoting [here](https://www.databricks.com/blog/2018/11/01/sql-pivot-converting-rows-to-columns.html)